# Interpolation to Spherical Grids in the Einstein Toolkit


<font color='blue'>**This module has not yet undergone validation testing.**</font>

## Structure

This module is designed to interpolate arbitrary quantities on [Einstein Toolkit](https://einsteintoolkit.org/) Adaptive-Mesh Refinement (AMR) grids (using the [Carpet](https://carpetcode.org/) AMR infrastructure) to numerical grids with spherical sampling.

Given some set of $N$ quantities $\mathbf{Q}=\{Q_0,Q_1,Q_2,...,Q_{N-2},Q_{N-1}\}$, this module performs the following for each $Q_i$:

1. Evaluate $Q_i$ at all gridpoints that are not ghost zones. Sometimes $Q_i$ is computed using finite difference derivatives, so this is necessary.
1. Call upon Carpet's interpolation and interprocessor synchronization functions to fill in $Q_i$ at all ghost zones, *except* at the outer boundary. We do not generally trust $Q_i$ at the outer boundary due to errors associated with the approximate outer boundary conditions. 
1. At this point, $Q_i$ is set at all gridpoints except ghost zones at the outer boundary. Interpolate $Q_i$ to the spherical grids, **maintaining the Cartesian basis for all vectors and tensors**, and append the result to a file.

In [1]:
!mkdir interp_sph_grids_ETK     2>/dev/null # 2>/dev/null: Don't throw an error or warning if the directory already exists.
!mkdir interp_sph_grids_ETK/src 2>/dev/null # 2>/dev/null: Don't throw an error or warning if the directory already exists.

### Low-Level ETK Interpolation Function

We start by writing the low-level interpolation function **Interpolate_to_sph_grid()**, which  to file. 

**Interpolate_to_sph_grid()** takes as input
* **cctkGH**: Information about the underlying Cactus/Carpet grid hierarchy.
* **interp_num_points**: Number of destination interpolation points
* **point_x_temp, point_y_temp, point_z_temp**: Cartesian $(x,y,z)$ location for each of the **interp_num_points** interpolation points.
* **input_array_names[1]**: List of input gridfunction names to interpolate. We will do this only one gridfunction at a time, for gridfunction $Q_i$, as described above.

**Interpolate_to_sph_grid()** outputs:
* **output_f[1]**: The gridfunction **input_array_names[1]** interpolated to the set of **interp_num_points** specified in the input.

In [2]:
%%writefile interp_sph_grids_ETK/src/Interpolate_to_sph_grid.h

void Interpolate_to_sph_grid(cGH *cctkGH,CCTK_INT interp_num_points, CCTK_INT interp_order,
                             CCTK_REAL *point_x_temp,CCTK_REAL *point_y_temp,CCTK_REAL *point_z_temp, 
                             const CCTK_STRING input_array_names[1], CCTK_REAL *output_f[1]) {
  DECLARE_CCTK_PARAMETERS; 
  CCTK_INT ierr;

  const CCTK_INT NUM_INPUT_ARRAYS=1;
  const CCTK_INT NUM_OUTPUT_ARRAYS=1;

  CCTK_STRING coord_system = "cart3d";

  // Set up handles
  const CCTK_INT coord_system_handle = CCTK_CoordSystemHandle(coord_system);
  if (coord_system_handle < 0) {
    CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
        "can't get coordinate system handle for coordinate system \"%s\"!",
               coord_system);
  }

  const CCTK_INT operator_handle = CCTK_InterpHandle(interpolator_name);
  if (operator_handle < 0)
    CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
               "couldn't find interpolator \"%s\"!",
               interpolator_name);

  char interp_order_string[10];
  snprintf(interp_order_string, 10, "order=%d", interp_order);
  CCTK_STRING interpolator_pars = interp_order_string;
  CCTK_INT param_table_handle = Util_TableCreateFromString(interpolator_pars);
  if (param_table_handle < 0) {
    CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
               "bad interpolator parameter(s) \"%s\"!",
               interpolator_pars);
  }
  
  CCTK_INT operand_indices[NUM_INPUT_ARRAYS]; //NUM_OUTPUT_ARRAYS + MAX_NUMBER_EXTRAS];
  for(int i = 0 ; i < NUM_INPUT_ARRAYS  ; i++) {
    operand_indices[i] = i;
  }
  Util_TableSetIntArray(param_table_handle, NUM_OUTPUT_ARRAYS,
                        operand_indices, "operand_indices");
  

  CCTK_INT opcodes[NUM_INPUT_ARRAYS];
  for(int i = 0 ; i < NUM_INPUT_ARRAYS  ; i++) {
    opcodes[i] = 0;
  }
  Util_TableSetIntArray(param_table_handle, NUM_OUTPUT_ARRAYS, 
                        opcodes, "opcodes");

  const void* interp_coords[3] 
    = { (const void *) point_x_temp,
        (const void *) point_y_temp,
        (const void *) point_z_temp };

  CCTK_INT input_array_indices[NUM_INPUT_ARRAYS];
  for(int i = 0 ; i < NUM_INPUT_ARRAYS ; i++) {
    input_array_indices[i] = CCTK_VarIndex(input_array_names[i]);
    if(input_array_indices[i] < 0) {
      CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
        "COULD NOT FIND VARIABLE '%s'.",
        input_array_names[i]);
      exit(1);
    }
  }

  CCTK_INT output_array_types[NUM_OUTPUT_ARRAYS];
  for(int i = 0 ; i < NUM_OUTPUT_ARRAYS ; i++) {
    output_array_types[i] = CCTK_VARIABLE_REAL;
  }

  void * output_arrays[NUM_OUTPUT_ARRAYS]
    = { (void *) output_f[0] };

  // actual interpolation call
  ierr = CCTK_InterpGridArrays(cctkGH,
                               3, // number of dimensions 
                               operator_handle,
                               param_table_handle,
                               coord_system_handle,
                               interp_num_points,
                               CCTK_VARIABLE_REAL,
                               interp_coords,
                               NUM_INPUT_ARRAYS, // Number of input arrays
                               input_array_indices,
                               NUM_OUTPUT_ARRAYS, // Number of output arrays
                               output_array_types,
                               output_arrays);
  if (ierr<0) {
    CCTK_WARN(1,"interpolation screwed up");
    Util_TableDestroy(param_table_handle);
    exit(1);
  }

  ierr = Util_TableDestroy(param_table_handle);
  if (ierr != 0) {
    CCTK_WARN(1,"Could not destroy table");
    exit(1);
  }
}

Overwriting interp_sph_grids_ETK/src/Interpolate_to_sph_grid.h


### Setting up the Spherical Grids


* By default, we set logarithmic radial coordinates: $r(x_{0,i}) = R_0 + e^{x_{0,i}}$, where
  + $x_{0,i} = x_{0, \mathrm{beg}} + \left(i+\frac{1}{2}\right) \Delta x_0$
  + $x_{0, {\mathrm{beg}}} = \log\left( R_{\mathrm{in}} - R_0 \right)$
  + $\Delta x_0 = \frac{1}{N_0}\log\left(\frac{R_\mathrm{out} - R_0}{R_\mathrm{in} - R_0}\right)$
* As for the polar angle $\theta$, there are two options:
  + **Option 1**: 
  $$
  \theta(x_{1,j})  \, = \, \theta_c \, + \, \left( \pi - 2 \theta_c \right) x_{1,j} \, 
+ \, \xi \, \sin\left(2 \pi x_{1,j} \right) \text{, where}
$$
    + $x_{1,j} = x_{1, \mathrm{beg}} + \left(j+\frac{1}{2}\right) \Delta x_1$
    + $\Delta x_1 = \frac{1}{N_1}$
  + **Option 2**: 
  $$
  \theta(x_{1,j}) = \frac{\pi}{2} \left[  1  + \left(1-\xi \right) \left(2 x_{1,j} - 1 \right) + \left( \xi - \frac{2 \theta_c}{\pi} \right) \left( 2 x_{1,j} - 1 \right)^n \right] \text{, where}
  $$
    + $n$ is odd
    + $x_{1,j} = x_{1, \mathrm{beg}} + \left(j+\frac{1}{2}\right) \Delta x_1$
    + $\Delta x_1 = \frac{1}{N_1}$
* The azimuthal angle $\phi$ is uniform, so that $\phi(x_{2,k}) = x_{2,k}$:
  + $x_{2,k} \in [0,2\pi]$
  + $x_{2,k} = x_{2, \mathrm{beg}} + \left(k+\frac{1}{2}\right)\Delta x_{2}$
  + $\Delta x_{2} = \frac{ 2 \pi }{N_2}$

In [3]:
%%writefile interp_sph_grids_ETK/src/Set_up_interp_points_on_sph_grid.h

void sph_grid_Interpolate_many_pts__set_interp_pts(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  CCTK_REAL dx0 = log( (Rout - R0) / (Rin - R0) ) / ((CCTK_REAL)N0);
  CCTK_REAL dx1 = 1.0 / ((CCTK_REAL)N1);
  CCTK_REAL dx2 = 1.0 / ((CCTK_REAL)N2);
  CCTK_REAL x0_beg = log( Rin - R0 );
  CCTK_INT which_pt = 0;
  for(CCTK_INT k=0;k<N2;k++) for(CCTK_INT j=0;j<N1;j++) for(CCTK_INT i=0;i<N0;i++) {
    CCTK_REAL x0_i = x0_beg + ((CCTK_REAL)i + 0.5)*dx0;
    CCTK_REAL rr = R0 + exp(x0_i);

    CCTK_REAL x1_j = x1_beg + ((CCTK_REAL)j + 0.5)*dx1;
    CCTK_REAL th = -1e300;
    if(theta_option == 1) {
       th = th_c + (M_PI - 2.0*th_c)*x1_j + xi*sin(2.0*M_PI*x1_j);
    } else if (theta_option == 2) {
       th = M_PI/2.0 * ( 1.0 + (1.0 - xi)*(2.0*x1_j - 1.0) + (xi - 2.0*th_c/M_PI)*pow(2.0*x1_j - 1.0 ,th_n) );
    } else {
       printf("Error: theta_option = %d NOT SUPPORTED.",theta_option);
       exit(1);
    }
    
    CCTK_REAL x2_k = x2_beg + ((CCTK_REAL)k + 0.5)*dx2;
    CCTK_REAL ph = x2_k;

    points_x[which_pt] = rr*sin(th)*cos(ph);
    points_y[which_pt] = rr*sin(th)*sin(ph);
    points_z[which_pt] = rr*cos(th);
    which_pt++;
  }
}

Overwriting interp_sph_grids_ETK/src/Set_up_interp_points_on_sph_grid.h


### Outputting to File

Since they take almost no space, we attach the entire metadata to each interpolated function that is output:

In [4]:
%%writefile interp_sph_grids_ETK/src/output_to_file.h

void output_to_file(CCTK_ARGUMENTS,CCTK_REAL *output_f[1]) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  char filename[100];
  sprintf (filename, "%s/interp_sph_grids.dat", out_dir);
  FILE *file = fopen (filename,"a+");
  if (! file) {
    CCTK_VWarn (1, __LINE__, __FILE__, CCTK_THORNSTRING,
                "interp_sph_grid__ET_thorn: Cannot open output file '%s'", filename);
    exit(1);
  }

  fwrite(&N0, sizeof(CCTK_INT), 1, file);
  fwrite(&R0, sizeof(CCTK_REAL), 1, file);
  fwrite(&Rin, sizeof(CCTK_REAL), 1, file);
  fwrite(&Rout, sizeof(CCTK_REAL), 1, file);

  fwrite(&N1, sizeof(CCTK_INT), 1, file);
  fwrite(&x1_beg, sizeof(CCTK_REAL), 1, file);
  fwrite(&theta_option, sizeof(CCTK_INT), 1, file);
  fwrite(&th_c, sizeof(CCTK_REAL), 1, file);
  fwrite(&xi, sizeof(CCTK_REAL), 1, file);
  fwrite(&th_n, sizeof(CCTK_INT), 1, file);

  fwrite(&N2, sizeof(CCTK_INT), 1, file);
  fwrite(&x2_beg, sizeof(CCTK_REAL), 1, file);

  CCTK_REAL magic_number = 1.130814081305130e-21;
  fwrite(&magic_number, sizeof(CCTK_REAL), 1, file);
  fwrite(&cctk_iteration, sizeof(CCTK_INT), 1, file);
  fwrite(&cctk_time, sizeof(CCTK_REAL), 1, file);
  for(CCTK_INT i=0;i<1;i++) {
    fwrite(output_f[i], sizeof(CCTK_REAL)*N0*N1*N2, 1, file);
  }

  fclose(file);
}

Overwriting interp_sph_grids_ETK/src/output_to_file.h


## The Main Interpolation Driver Function

The **Interpolate_to_sph_grid_main_function()** function calls the above functions as follows:
1. **sph_grid_Interpolate_many_pts__set_interp_pts()**: First set up the spherical grids
1. **Interpolate_to_sph_grid()**: Output

In [5]:
%%writefile interp_sph_grids_ETK/src/main_function.cc

// Include needed ETK & C library header files:
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <ctype.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "cctk_Functions.h"
#include "util_Table.h"
#include "util_String.h"

// Include locally-defined C++ functions:
#include "Set_up_interp_points_on_sph_grid.h"
#include "Interpolate_to_sph_grid.h"
#include "output_to_file.h"

void Interpolate_to_sph_grid_main_function(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // Perform interpolation only at iteration == interp_out_iteration:
  if(cctk_iteration != interp_out_iteration) return;

  // Set up spherically sampled interpolation grid arrays points_x,points_y,points_z: 
  sph_grid_Interpolate_many_pts__set_interp_pts(CCTK_PASS_CTOC);

    
    
  // Set up output array:
  CCTK_REAL *output_f[1];
  output_f[0] = output_interped;
  // The name of the input gridfunction is always "interp_sph_grids_ETK::interped_gf":
  const CCTK_STRING input_array_names[1] = { "interp_sph_grids_ETK::interped_gf" };

  // Perform interpolation!
    int order=1;
  Interpolate_to_sph_grid(cctkGH, N0*N1*N2, order,
                             points_x,points_y,points_z, input_array_names, output_f);

  if(CCTK_MyProc(cctkGH)==0) {
    output_to_file(CCTK_PASS_CTOC,output_f);
    printf("Interpolate_to_sph_grid_main_function(): Just output to file at iteration %d\n",cctk_iteration);
  } else {
    printf("Interpolate_to_sph_grid_main_function(): Process !=0 waiting for file output at iteration %d\n",cctk_iteration);
  }
}

Overwriting interp_sph_grids_ETK/src/main_function.cc


# Use NRPy+ C Output to Set All Output Gridfunctions


In [7]:
# Step 1: Import needed NRPy+ parameters
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import sympy as sp
import NRPy_param_funcs as par
import loop

par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

from collections import namedtuple
gf_interp = namedtuple('gf_interp', 'description order')
gf_interp_list = []

interped_gf = gri.register_gridfunctions("AUX","interped_gf")

def interp_fileout(which_InterpCounter, expression, filename):
    kernel = fin.FD_outputC("returnstring",lhrh(lhs=gri.gfaccess("out_gfs","interped_gf"),rhs=expression),"outCverbose=False")
    output_type="a"
    if which_InterpCounter == 0:
        output_type="w"
    
    with open(filename, output_type) as file:
        file.write("if(*InterpCounter == "+str(which_InterpCounter)+") {\n")
        file.write(loop.loop(["i2","i1","i0"],
                             ["cctk_nghostzones[2]","cctk_nghostzones[1]","cctk_nghostzones[0]"],\
                             ["cctk_lsh[2]-cctk_nghostzones[2]",
                              "cctk_lsh[1]-cctk_nghostzones[1]",
                              "cctk_lsh[0]-cctk_nghostzones[0]"],\
                             ["1","1","1"],\
                             ["#pragma omp parallel for","",""],"   ",kernel))
        file.write("}\n")
    # If successful, return incremented which_InterpCounter:
    return which_InterpCounter+1

## Define gridfunctions to interpolate

In [8]:
NRPyoutfilename = "interp_sph_grids_ETK/src/list_of_functions_to_interpolate.h"

which_InterpCounter = 0

gf_interp_list.append(gf_interp("IGM density primitive",order="1"))
rho_b       = gri.register_gridfunctions("AUX","rho_b")
interp_expr = rho_b
which_InterpCounter = interp_fileout(which_InterpCounter,interp_expr,NRPyoutfilename)

gf_interp_list.append(gf_interp("IGM pressure primitive",order="1"))
P = gri.register_gridfunctions("AUX","P")
interp_expr = P
which_InterpCounter = interp_fileout(which_InterpCounter,interp_expr,NRPyoutfilename)

### Compute all 10 components of the 4-metric $g_{\mu\nu}$

We are given $\gamma_{ij}$, $\alpha$, and $\beta^i$ from ADMBase, and the 4-metric is given in terms of these quantities as
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$

In [9]:
# INPUT GRIDFUNCTIONS: The AUX or EVOL designation is *not* used in diagnostic modules.
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01")
betaU = ixp.register_gridfunctions_for_single_rank1("AUX","betaU")
alpha = gri.register_gridfunctions("AUX","alpha")

DIM=3
# Eq. 2.121 in B&S
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += gammaDD[i][j]*betaU[j]

# Now compute the beta contraction.
beta2 = sp.sympify(0)
for i in range(DIM):
    beta2 += betaU[i]*betaD[i]

# Eq. 2.122 in B&S
g4DD = ixp.zerorank2(DIM=4)
g4DD[0][0] = -alpha**2 + beta2
for i in range(DIM):
    g4DD[i+1][0] = g4DD[0][i+1] = betaD[i]
for i in range(DIM):
    for j in range(DIM):
        g4DD[i+1][j+1] = gammaDD[i][j]

for mu in range(4):
    for nu in range(mu,4):
        gf_interp_list.append(gf_interp("4-metric component g4DD"+str(mu)+str(nu),order="4"))
        interp_expr = g4DD[mu][nu]
        which_InterpCounter = interp_fileout(which_InterpCounter,interp_expr,NRPyoutfilename)

### Compute all 40 4-Christoffels $\Gamma^{\mu}_{\nu\delta}$

By definition,
$$
\Gamma^{\mu}_{\nu\delta} = \frac{1}{2} g^{\mu\eta} \left(g_{\eta\nu,\delta} + g_{\eta\delta,\nu} - g_{\nu\delta,\eta}  \right)
$$

Recall that $g_{\mu\nu}$ is given from $\gamma_{ij}$, $\alpha$, and $\beta^i$ via
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$

The derivatives $g_{\mu\nu,\eta}$ are then computed in terms of finite-difference derivatives of the input ADM gridfunctions $\gamma_{ij}$, $\alpha$, and $\beta^i$, **assuming that the 4-metric is static, so that $\partial_t g_{\mu\nu}=0$ for all $\mu$ and $\nu$**.

To compute $g^{\mu\nu}$, we use the standard formula (Eq. 4.49 in [Gourgoulhon](https://arxiv.org/pdf/gr-qc/0703035.pdf)):
$$
g^{\mu\nu} = \begin{pmatrix} 
-\frac{1}{\alpha^2} & \frac{\beta^i}{\alpha^2} \\
\frac{\beta^i}{\alpha^2} & \gamma^{ij} - \frac{\beta^i\beta^j}{\alpha^2}
\end{pmatrix},
$$
where $\gamma^{ij}$ is given by the inverse of $\gamma_{ij}$.

In [10]:
betaDdD = ixp.zerorank2()
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01")
betaU_dD   = ixp.declarerank2("betaU_dD","nosym")
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            # Recall that betaD[i] = gammaDD[i][j]*betaU[j] (Eq. 2.121 in B&S)
            betaDdD[i][k] += gammaDD_dD[i][j][k]*betaU[j] + gammaDD[i][j]*betaU_dD[j][k]

# Eq. 2.122 in B&S
g4DDdD = ixp.zerorank3(DIM=4)
alpha_dD   = ixp.declarerank1("alpha_dD")
for i in range(DIM):
    # Recall that g4DD[0][0] = -alpha^2 + betaU[i]*betaD[i]
    g4DDdD[0][0][i+1] += -2*alpha*alpha_dD[i] 
    for j in range(DIM):
        g4DDdD[0][0][i+1] += betaU_dD[j][i]*betaD[j] + betaU[j]*betaDdD[j][i]

for i in range(DIM):
    for j in range(DIM):
        # Recall that g4DD[i][0] = g4DD[0][i] = betaD[i]
        g4DDdD[i+1][0][j+1] = g4DDdD[0][i+1][j+1] = betaDdD[i][j]
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            # Recall that g4DD[i][j] = gammaDD[i][j]
            g4DDdD[i+1][j+1][k+1] = gammaDD_dD[i][j][k]

gammaUU, dummyDET = ixp.symm_matrix_inverter3x3(gammaDD)

g4UU = ixp.zerorank2(DIM=4)
g4UU[0][0] = -1 / alpha**2
for i in range(DIM):
    g4UU[0][i+1] = g4UU[i+1][0] = betaU[i]/alpha**2
for i in range(DIM):
    for j in range(DIM):
        g4UU[i+1][j+1] = gammaUU[i][j] - betaU[i]*betaU[j]/alpha**2

Again, we are to compute:
$$
\Gamma^{\mu}_{\nu\delta} = \frac{1}{2} g^{\mu\eta} \left(g_{\eta\nu,\delta} + g_{\eta\delta,\nu} - g_{\nu\delta,\eta}  \right)
$$

In [11]:
Gamma4UDD = ixp.zerorank3(DIM=4)
for mu in range(4):
    for nu in range(4):
        for delta in range(4):
            for eta in range(4):
                Gamma4UDD[mu][nu][delta] += sp.Rational(1,2)*g4UU[mu][eta]*\
                (g4DDdD[eta][nu][delta] + g4DDdD[eta][delta][nu] - g4DDdD[nu][delta][eta])

# Now output the 4-Christoffels to file:
for mu in range(4):
    for nu in range(4):
        for delta in range(nu,4):
            gf_interp_list.append(gf_interp("4-Christoffel GammaUDD"+str(mu)+str(nu)+str(delta),order="4"))
            interp_expr = Gamma4UDD[mu][nu][delta]
            which_InterpCounter = interp_fileout(which_InterpCounter,interp_expr,NRPyoutfilename)

### Next we write the function for the above NRPy+ output

In [26]:
%%writefile interp_sph_grids_ETK/src/interpolate_single_function.cc
#include <stdio.h>
#include <stdlib.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

// Set the gridfunction interped_gf, according to the interpolation counter variable interp_counter.
//    For example, we might interpolate "IllinoisGRMHD::rho_b" if interp_counter==0. The following
//    function takes care of these
void list_of_functions_to_interpolate(cGH *cctkGH,const CCTK_INT *cctk_lsh,const CCTK_INT *cctk_nghostzones,
                                     const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                                     const CCTK_INT *InterpCounter,
                                     const CCTK_REAL *rho_bGF,const CCTK_REAL *PGF,
                                     const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,
                                     const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                                     const CCTK_REAL *betaU0GF,const CCTK_REAL *betaU1GF,const CCTK_REAL *betaU2GF,
                                     const CCTK_REAL *alphaGF,   CCTK_REAL *interped_gfGF) {
#include "list_of_functions_to_interpolate.h"
}

void Interpolate_single_function(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  const CCTK_REAL invdx0 = 1.0 / CCTK_DELTA_SPACE(0);
  const CCTK_REAL invdx1 = 1.0 / CCTK_DELTA_SPACE(1);
  const CCTK_REAL invdx2 = 1.0 / CCTK_DELTA_SPACE(2);
  list_of_functions_to_interpolate(cctkGH,cctk_lsh,cctk_nghostzones,invdx0,invdx1,invdx2,
                                   InterpCounter,
                                   rho_b,P,
                                   gxx,gxy,gxz,gyy,gyz,gzz,
                                   betax,betay,betaz,alp, interped_gf);
}

Overwriting interp_sph_grids_ETK/src/interpolate_single_function.cc


### Interpolation Function Counter Routines, for Scheduling

In [23]:
with open("interp_sph_grids_ETK/src/interp_counter.cc", "w") as file:
    file.write("#define NumInterpFunctions "+str(which_InterpCounter)+"\n")

In [24]:
%%writefile -a interp_sph_grids_ETK/src/interp_counter.cc
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <ctype.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void SphGrid_InitializeInterpCounterToZero(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  *InterpCounter = 0;

  if(verbose==2) printf("interp_sph_grids_ETK: Just set InterpCounter to %d\n",*InterpCounter);
}

void SphGrid_GRMHD_InitializeInterpCounter(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  if(cctk_iteration == interp_out_iteration) {
    *InterpCounter = NumInterpFunctions;
    if(verbose==2) printf("interp_sph_grids_ETK: Just set InterpCounter to %d == NumInterpFunctions\n",
                          *InterpCounter);
  }
}

void SphGrid_GRMHD_DecrementInterpCounter(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

    (*InterpCounter)--;
    if(verbose==2) printf("interp_sph_grids_ETK: Just decremented InterpCounter to %d\n",*InterpCounter);
}

Appending to interp_sph_grids_ETK/src/interp_counter.cc


In [13]:
%%writefile interp_sph_grids_ETK/src/make.code.defn
# Main make.code.defn file for thorn interp_sph_grids_ETK

# Source files in this directory
SRCS =  main_function.cc interp_counter.cc interpolate_single_function.cc

Overwriting interp_sph_grids_ETK/src/make.code.defn


## CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns.
1. $\text{param.ccl}$: specifies free parameters within the thorn.
1. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions.

Let's start with $\text{interface.ccl}$. The [official Einstein Toolkit (Cactus) documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManual.html) defines what must/should be included in an interface.ccl file [**here**](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 

In [14]:
%%writefile interp_sph_grids_ETK/interface.ccl

# With "implements", we give our thorn its unique name.
implements: interp_sph_grids_ETK

# By "inheriting" other thorns, we tell the Toolkit that we 
#   will rely on variables/function that exist within those
#   functions. 
inherits:   admbase IllinoisGRMHD Grid

# Tell the Toolkit that we want "interped_gf" and "InterpCounter"
#    and invariants to NOT be visible to other thorns, by using 
#    the keyword "private". Note that declaring these 
#    gridfunctions here *does not* allocate memory for them;
#    that is done by the schedule.ccl file.
private:
CCTK_REAL interpolation_gf type=GF timelevels=1 tags='InterpNumTimelevels=1 prolongation="none" Checkpoint="no"'
{
  interped_gf
} "Gridfunction containing output from interpolation."

int InterpCounterVar type = SCALAR tags='checkpoint="no"'
{
  InterpCounter
} "Counter that keeps track of which function we are interpolating."

CCTK_REAL interp_pointcoords_and_output_arrays TYPE=ARRAY DISTRIB=CONSTANT DIM=1 SIZE=N0*N1*N2
{
  points_x,points_y,points_z,
  output_interped
}

Overwriting interp_sph_grids_ETK/interface.ccl


We will now write the file $\text{param.ccl}$. This file allows the listed parameters to be set at runtime. We also give allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). 

In [15]:
%%writefile interp_sph_grids_ETK/param.ccl

# Output the interpolated data to the IO::out_dir directory:
shares: IO
USES STRING out_dir

restricted:
    
########################################
# BASIC THORN STEERING PARAMETERS
CCTK_INT interp_out_iteration "Which iteration to interpolate to spherical grids?" STEERABLE=RECOVER
{
  0:* :: ""
} 960000

## Interpolator information
CCTK_STRING interpolator_name "Which interpolator to use?" STEERABLE=RECOVER
{
  ".+" :: "Any nonempty string; an unsupported value will throw an error."
} "Lagrange polynomial interpolation"

CCTK_INT verbose "Set verbosity level: 1=useful info; 2=moderately annoying (though useful for debugging)" STEERABLE=ALWAYS
{
  0:2 :: "0 = no output; 1=useful info; 2=moderately annoying (though useful for debugging)"
} 1
########################################
# SPHERICAL COORDINATE SYSTEM PARAMETERS
CCTK_INT N0 "Number of points in r direction" STEERABLE=RECOVER
{
  0:* :: ""
} 96

CCTK_INT N1 "Number of points in theta direction" STEERABLE=RECOVER
{
  0:* :: ""
} 96

CCTK_INT N2 "Number of points in phi direction" STEERABLE=RECOVER
{
  0:* :: ""
} 96

##########
# Radial parameters:
CCTK_REAL R0 "Radial offset: r(x0) = R_0 + exp(x0). Probably should keep it set to zero." STEERABLE=RECOVER
{
  0:* :: ""
} 0.0

CCTK_REAL Rin "x0 offset: x0 = log(Rin-R0) + (i + 0.5)Dx0." STEERABLE=RECOVER
{
  0:* :: ""
} 1.08986052555408

CCTK_REAL Rout "Dx0 = log( (Rout-R0) / (Rin-R0) )/N0" STEERABLE=RECOVER
{
  0:* :: ""
} 80.0

##########
# Theta parameters:
CCTK_REAL x1_beg "x1 offset: x1 = x1_beg + (j + 0.5)Dx1. Probably should keep it set to zero." STEERABLE=RECOVER
{
  0:* :: ""
} 0.0

CCTK_INT theta_option "Which prescription for theta should be used? 1 or 2?" STEERABLE=RECOVER
{
  1:2 :: ""
} 1

CCTK_REAL th_c "theta_c: Angular cutout size for theta = 0 and pi" STEERABLE=RECOVER
{
  0:* :: ""
} 0.053407075111026485 # 0.017*pi

CCTK_REAL xi "Amplitude of nonlinear part of the theta distribution." STEERABLE=RECOVER
{
  0:* :: ""
} 0.25

CCTK_INT th_n "Power of nonlinear part of theta distribution. Only for theta_option=2" STEERABLE=RECOVER
{
  0:* :: ""
} 9

##########
# Phi parameters:
CCTK_REAL x2_beg "x2 offset: x2 = x2_beg + (k + 0.5)Dx2. Probably should keep it set to zero." STEERABLE=RECOVER
{
  0:* :: ""
} 0.0
########################################

Overwriting interp_sph_grids_ETK/param.ccl


Finally, we will write the file $\text{schedule.ccl}$; its official documentation is found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

This file declares storage for variables declared in the $\text{interface.ccl}$ file and specifies when the various parts of the thorn will be run:

In [16]:
%%writefile interp_sph_grids_ETK/schedule.ccl

STORAGE: interpolation_gf[1]
STORAGE: InterpCounterVar
STORAGE: interp_pointcoords_and_output_arrays

Overwriting interp_sph_grids_ETK/schedule.ccl
